In [2]:
import gurobipy as gp
from gurobipy import GRB
from operator import itemgetter
import itertools
import math
import numpy as np 
import json
from prettytable import PrettyTable

In [3]:
def formar_asignaciones(profesores_por_curso: dict):
    lista_asignaciones = []
    for curso in profesores_por_curso:
        for llave in profesores_por_curso[curso]:
            for tupla in profesores_por_curso[curso][llave]:
                lista_asignaciones.append((curso, tupla[0], tupla[1]))
    return lista_asignaciones

def formar_asignaciones_multiples(clase_conjunta):
    lista_asignaciones_multiples = []
    cantidad_cursos_simultaneos = {}
    for cs,p,k in clase_conjunta:
        for c in cs:
            lista_asignaciones_multiples.append((c,p,k))
            cantidad_cursos_simultaneos[c,p,k] = len(cs)
    return lista_asignaciones_multiples, cantidad_cursos_simultaneos

def cantidad_maxima_de_dias(carga_profesores, ramos_por_curso):
    tope_dias = {}
    for profesor in carga_profesores:
        suma = 0
        for cantidad_cursos in carga_profesores[profesor]:
            for curso,clase in carga_profesores[profesor][cantidad_cursos]:
                suma += ramos_por_curso[curso][clase] * int(cantidad_cursos)
            suma += int(len(carga_profesores[profesor][cantidad_cursos]) / int(cantidad_cursos))
        if suma <= 3:
            tope_dias[profesor] = 1
        elif suma <= 6:
            tope_dias[profesor] = 2
        elif suma <= 9:
            tope_dias[profesor] = 3
        elif suma <= 12:
            tope_dias[profesor] = 4
        else:
            tope_dias[profesor] = 5

    return tope_dias

In [4]:
model = gp.Model("Generación de horarios de colegio")
# model.setParam('OutputFlag', 1)
f = open('parametros3.json',encoding='utf8')

M = 10000
data = json.load(f)
dias = data['dias']
modulos = data['modulos']
ramos_con_modulos_seguidos = data['ramos_con_modulos_seguidos']
combinaciones = data['combinaciones']
horarios = data['horarios']
profesores_por_curso = data['profesores_por_curso']
ramos_por_curso = data['ramos_por_curso']
carga_profesores = data['carga_profesores']
profesores = list(carga_profesores.keys())
multiples_cursos = data['multiples_cursos']
multiples_profesores = data['multiples_profesores']
modulos_consecutivos = [(a,b) for a in modulos for b in modulos if a+1 == b]
# multiples_cursos_y_profesores = data['multiples_cursos_y_profesores']

# # # ! DATOS HECHIZOS
# cantidad_profesores_ramo = {
#     "Lenguaje": 10,
#     "Matemáticas": 10,
#     "Historia": 7,
#     "Artes_Visuales": 5,
#     "Música": 5,
#     "Educación_Física": 8,
#     "Orientación": 4,
#     "Tecnología": 4,
#     "Religión": 5,
#     "Ciencias_Naturales": 7,
#     "Inglés": 7,
#     "Consejo_de_Curso": 1,
#     "Filosofía": 1
# }
# cantidad_cursos_profesor = {
#     "Lenguaje": 4,
#     "Matemáticas": 4,
#     "Historia": 4,
#     "Artes_Visuales": 6,
#     "Música": 6,
#     "Educación_Física": 4,
#     "Orientación": 10,
#     "Tecnología": 10,
#     "Religión": 7,
#     "Ciencias_Naturales": 4,
#     "Inglés": 4,
#     "Consejo_de_Curso": 10,
#     "Filosofía": 10
# }

# import simulador
# ramos_por_curso = simulador.generar_horas_libres(ramos_por_curso)
# lista_profesores_para_asignaciones, profesores = simulador.generar_profesores(7, 74, cantidad_profesores_ramo, cantidad_cursos_profesor)
# profesores_por_curso = simulador.generar_asignaciones(lista_profesores_para_asignaciones, ramos_por_curso)

asignaciones_normales = formar_asignaciones(profesores_por_curso)
# asignaciones_multiples, cantidad_de_cursos_simultaneos = formar_asignaciones_multiples(multiples_cursos)
tope_dias = cantidad_maxima_de_dias(carga_profesores, ramos_por_curso)

asignaciones = asignaciones_normales

x = model.addVars(asignaciones,dias,modulos,vtype=GRB.BINARY, name="x")
s = model.addVars(asignaciones,dias,vtype=GRB.BINARY, name="s") # * Solo para los cursos donde se prefieren módulos dobles
y = model.addVars(profesores,dias,vtype=GRB.BINARY,name="y") # * Solo para los profesores que tienen menos de 5 días

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-17


##### Se tiene solo una clase a la vez y se tiene una clase en cada módulo del horario (se consideran los casos donde una clase tiene más de un profesor)

In [5]:
model.addConstrs((sum(x[c,p,k,d,j] for p,k in profesores_por_curso[c]['1']) == 1 \
    for c in horarios for d in horarios[c] for j in horarios[c][d]),name="R1");

In [6]:
model.addConstrs((sum(x[c,p,k,d,j] for p,k in profesores_por_curso[c]['M']) >= 1 \
    for c in horarios for d in horarios[c] for j in horarios[c][d]),name="R1b");

KeyError: 'M'

In [7]:
for c,ps,k in multiples_profesores:
    model.addConstrs((x[c,p1,k,d,j] - x[c,p2,k,d,j] == 0
    for p1,p2 in [(ps[a],ps[b]) for a in range(len(ps)) for b in range(len(ps)) if a < b]
    for d in dias for j in modulos),name="R1c");

##### En el horario se tiene exactamente la cantidad de clases definida

In [8]:
model.addConstrs((sum(x[c,p,k,d,j] for d in dias for j in modulos) == ramos_por_curso[c][k] \
    for c in profesores_por_curso for l in profesores_por_curso[c] for p,k in profesores_por_curso[c][l]),name="R2");

##### No se tiene clases en los módulos que no son parte del horario

In [10]:
model.addConstrs((sum(x[c,p,k,d,j] for l in profesores_por_curso[c] for p,k in profesores_por_curso[c][l]) == 0 \
    for c in horarios for d in horarios[c] for j in list(set(modulos) - set(horarios[c][d]))),name="R3");

##### Asignación profesores

In [11]:
for cs,p2,k2 in multiples_cursos :
     model.addConstrs((sum(x[c,p,k,d,j] for c,k in carga_profesores[p][q] if c != c2) + \
          x[c2,p,k2,d,j] <= int(q) \
          for c2 in [cs[a] for a in range(len(cs)) for b in range(len(cs)) if a < b]
          for p in list(carga_profesores.keys()) for q in carga_profesores[p] for d in dias for j in modulos if p == p2),name="R4");

In [12]:
model.addConstrs((sum(x[c,p,k,d,j] for c,k in carga_profesores[p][q]) <= int(q) \
     for p in list(carga_profesores.keys()) for q in carga_profesores[p] for d in dias for j in modulos),name="R5");

In [13]:
for cs,p,k in multiples_cursos :
    model.addConstrs((x[c1,p,k,d,j] - x[c2,p,k,d,j] == 0 \
        for c1,c2 in [(cs[a],cs[b]) for a in range(len(cs)) for b in range(len(cs)) if a < b]
        for d in dias for j in modulos),name="R6");

In [14]:
model.addConstrs((sum(x[c,p,k,d,j] for j in modulos) <= 2 for d in dias for c in horarios 
for l in profesores_por_curso[c] for p,k in profesores_por_curso[c][l]),name="R7");

In [15]:
model.addConstrs((x[c,p,k,d,j] + x[c,p,k,d,z] <= 1 for c,p,k in asignaciones for d in dias \
    for j,z in list(set([(a,b) for a in horarios[c][d] for b in horarios[c][d] if a < b]) - set([tuple(comb) for comb in combinaciones]))),name="R8");

In [16]:
model.addConstrs((M * s[c,p,k,d] >= sum(x[c,p,k,d,j] for j in modulos) for c,p,k in asignaciones for d in dias),name="RS1");

In [17]:
model.addConstrs((sum(s[c,p,k,d] for d in dias) <= math.ceil(ramos_por_curso[c][k] / 2) \
    for c in horarios for l in profesores_por_curso[c] for p,k in profesores_por_curso[c][l] if k in ramos_con_modulos_seguidos),name="RS2");

##### Tope de días según la cantidad de horas de clases que hace cada profesor

In [18]:
model.addConstrs((M * y[p,d] >= sum(s[c,p,k,d] for c,k in carga_profesores[p][q]) \
    for d in dias for p in carga_profesores for q in carga_profesores[p]),name="RY1");

In [19]:
model.addConstrs((sum(y[p,d] for d in dias) <= tope_dias[p] for p in carga_profesores),name="RY2");

In [20]:
obj = sum(x[c,p,k,d,j] + x[c2,p2,k2,d,z] for c,p,k in asignaciones for c2,p2,k2 in asignaciones for d in dias for j,z in modulos_consecutivos
if p == p2 and c == c2)
model.setObjective(obj, GRB.MINIMIZE)

model.write('model.lp')
model.setParam("PoolSolutions", 10)
model.setParam('PoolSearchMode', 2)
model.setParam('OutputFlag', 1)
model.update()

# model.computeIIS()
# removed =[]
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s' % c.constrName)
#         # Remove a single constraint from the model
#         removed.append(str(c.constrName))
#         model.remove(c)
model.optimize()
for i in range(model.SolCount):
    model.setParam("SolutionNumber", i)
    model.update()

    print(model.optimize())
    model.write(f"solutions/out{i+1}.sol")
# model.optimize()
# model.write('out.sol')

Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 89788 rows, 34260 columns and 313744 nonzeros
Model fingerprint: 0x6c784eed
Variable types: 0 continuous, 34260 integer (34260 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 64286 rows and 6738 columns
Presolve time: 1.49s
Presolved: 25502 rows, 27522 columns, 172103 nonzeros
Variable types: 0 continuous, 27522 integer (27522 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   37397    3.8609650e+03   1.539373e+02   1.108695e+10      5s
   55245    3.8597365e+03   1.833719e+01   5.823224e+09     10s
   65273    3.8163882e+03   

In [21]:
def leer_output(numero_sol):
    lista_final = []
    with open(f'solutions/out{numero_sol}.sol', 'r') as file:
        lines = file.readlines()[2:]
        for line in lines:
            if line[-2] == '1':
                if line[-4] == ']':
                    line = line.strip('\n')
                    lista_final.append(line)
                else:
                    line = line.strip('\n')
                    if round(float(line[line.find(']') + 2:])) == 1:
                        lista_final.append(line)
            else:
                if line[-4] != ']':
                    line = line.strip('\n')
                    if int(float(line[line.find(']') + 2:])) == 1:
                        lista_final.append(line)
    diccionario_dias = {
        'Lunes': 1,
        'Martes': 2,
        'Miércoles': 3,
        'Jueves': 4,
        'Viernes': 5
    }

    vuelta = {
        1: 'Lunes',
        2: 'Martes',
        3: 'Miércoles',
        4: 'Jueves',
        5: 'Viernes'
    }

    profesores_final = []
    for elem in lista_final:
        if elem[0] == "x":
            elem = elem[2:-3]
            elem = elem.split(',')
            profesores_final.append(elem)
    for i in range(len(profesores_final)):
        profesores_final[i][3] = diccionario_dias[profesores_final[i][3]]
    profesores_final = sorted(profesores_final, key= lambda x: x[4])
    profesores_final = sorted(profesores_final, key= lambda x: x[3])
    profesores_final = sorted(profesores_final, key= lambda x: x[0])

    for i in range(len(profesores_final)):
        profesores_final[i][3] = vuelta[profesores_final[i][3]]

    ultimo_curso = None
    ultimo_ramo = None
    ultimo_dia = None
    ultimo_modulo = None
    indices = []
    for i in range(len(profesores_final)):
        if profesores_final[i][0] == ultimo_curso:
            if profesores_final[i][2] == ultimo_ramo:
                if profesores_final[i][3] == ultimo_dia:
                    if profesores_final[i][4] == ultimo_modulo:
                        indices.append(i)
        ultimo_curso = profesores_final[i][0]
        ultimo_ramo = profesores_final[i][2]
        ultimo_dia = profesores_final[i][3]
        ultimo_modulo = profesores_final[i][4]
    indices.reverse()
    for indice in indices:
        profesores_final[indice - 1][1] = profesores_final[indice - 1][1] + ' & ' + profesores_final[indice][1]
        profesores_final.pop(indice)
    return profesores_final

def generar_horario(curso, numero_sol):

    clases_curso = []
    dias_semana = []
    ultimo_dia = None
    horario = PrettyTable(dias_semana)
    lista_auxiliar = []
    lista_dia = None
    for elem in leer_output(numero_sol):    
        if elem[0] == curso:
            clases_curso.append([elem[2], elem[3], elem[4], elem[1]])
            if elem[3] not in dias_semana:
                dias_semana.append(elem[3])
            if elem[3] != ultimo_dia:
                lista_auxiliar.append(lista_dia)
                lista_dia = []
                ultimo_dia = elem[3]
                lista_dia.append(ultimo_dia)
                elem[2] = elem[2].replace('_', ' ')
                elem[1] = elem[1].replace('_', ' ')
                lista_dia.append(f'{elem[2]} \n {elem[1]}')
            else:
                elem[2] = elem[2].replace('_', ' ')
                elem[1] = elem[1].replace('_', ' ')
                lista_dia.append(f'{elem[2]} \n {elem[1]}')

    lista_auxiliar.append(lista_dia)
    lista_auxiliar = lista_auxiliar[1:]
    len_maxima = 0
    for dia in lista_auxiliar:
        if len(dia) > len_maxima:
            len_maxima = len(dia)

    horario.add_column('', [i+1 for i in range(len_maxima - 1)])
    
    for i in range(len(lista_auxiliar)):
        for j in range(len_maxima - len(lista_auxiliar[i])):
            lista_auxiliar[i].append('')
        horario.add_column(lista_auxiliar[i][0], lista_auxiliar[i][1:], align='c')
    horario.format = True   
    print(f'Horario del curso {curso}: ')   
    return horario

curso = 'IVA'
generar_horario(curso,numero_sol=8)

Horario del curso IVA: 


,Lunes,Martes,Miércoles,Jueves,Viernes
1,Física Orvyxhyd,Física Orvyxhyd,G2 Nncolmnk,Lenguaje Dwivygdq,Física Orvyxhyd
2,Alemán Yiqzsxsm,Alemán Yiqzsxsm,Consejo de Curso Orvyxhyd,Alemán Yiqzsxsm,Historia Weofvsvu
3,Geografía Svixsrbd,Alemán Yiqzsxsm,G2 Nncolmnk,Inglés Jumijrpx,Historia Weofvsvu
4,Artes Visuales Vkmfgwym,G3 Hgcsbhrt,Biología Pwekysoi,G3 Hgcsbhrt,Matemáticas Cdinnylg
5,Artes Visuales Vkmfgwym,Matemáticas Cdinnylg,Lenguaje Dwivygdq,Historia Weofvsvu,Matemáticas Cdinnylg
6,Educación Física Ncnqafso,Inglés Jumijrpx,Lenguaje Dwivygdq,Ciencias Sociales Cjntorhr,Lenguaje Dwivygdq
7,Educación Física Ncnqafso,G2 Nncolmnk,Alemán Yiqzsxsm,Biología Pwekysoi,Biología Pwekysoi
8,,Geografía Svixsrbd,Geografía Svixsrbd,Matemáticas Cdinnylg,
9,,Ciencias Sociales Cjntorhr,Inglés Jumijrpx,G2 Nncolmnk,
10,,,Inglés Jumijrpx,,


In [37]:
output = {}
ultimo_elem = None
ultimo_dia = None
ultimo_modulo = None
for elem in leer_output(numero_sol=1):
    if elem[0] == ultimo_elem:
        if elem[3] == ultimo_dia:
            if elem[4] == ultimo_modulo:
                print(elem)
                output[elem[0]][elem[3]][elem[4]] += ',' + elem[1]
            elif elem[4] != ultimo_modulo:
                output[elem[0]][elem[3]][elem[4]] = elem[2] + ',' + elem[1]
        elif elem[3] != ultimo_dia:
            output[elem[0]][elem[3]] = {}
            output[elem[0]][elem[3]][elem[4]] = elem[2] + ',' + elem[1]
    else:
        output[elem[0]] = {}
        output[elem[0]][elem[3]] = {}
        output[elem[0]][elem[3]][elem[4]] = elem[2] + ',' + elem[1]
    ultimo_elem = elem[0]
    ultimo_dia = elem[3]
    ultimo_modulo = elem[4]
output

{'1A': {'Lunes': {'1': 'Matemáticas,Bzbkyeqh',
   '2': 'Matemáticas,Bzbkyeqh',
   '3': 'Alemán,Eieyagjc',
   '4': 'Música,Pxymekhf',
   '5': 'Lenguaje,Zjszbjef',
   '6': 'Tecnología,Mlvafpyt',
   '7': 'Tecnología,Mlvafpyt'},
  'Martes': {'1': 'Matemáticas,Bzbkyeqh',
   '2': 'Alemán,Eieyagjc',
   '3': 'Alemán,Eieyagjc',
   '4': 'Ciencias_Naturales,Ncnqafso',
   '5': 'Lenguaje,Zjszbjef',
   '6': 'Historia,Nipudbji',
   '7': 'Orientación,Wrbjwpou',
   '8': 'Taller_Comprensión_Lectora,Atzgrmcj'},
  'Miércoles': {'1': 'Religión,Bzbkyeqh',
   '2': 'Historia,Nipudbji',
   '3': 'Matemáticas,Bzbkyeqh',
   '4': 'Educación_Física,Yiqzsxsm',
   '5': 'Educación_Física,Yiqzsxsm',
   '6': 'Lenguaje,Zjszbjef',
   '7': 'Lenguaje,Zjszbjef',
   '8': 'Ciencias_Naturales,Ncnqafso'},
  'Jueves': {'1': 'Matemáticas,Bzbkyeqh',
   '2': 'Lenguaje,Zjszbjef',
   '3': 'Lenguaje,Zjszbjef',
   '4': 'Alemán,Eieyagjc',
   '5': 'Alemán,Eieyagjc',
   '6': 'Artes_Visuales,Ycaeozob',
   '7': 'Artes_Visuales,Ycaeozob',
   

In [ ]:
def generar_horario_profesor(nombre_profesor):
    clases_profesor = []
    dict_dias = {'Lunes': 1, 'Martes': 2, 'Miércoles': 3, 'Jueves': 4, 'Viernes': 5, 'Sábado': 6, 'Domingo': 7}
    dict_dias_inverso = {1: 'Lunes', 2: 'Martes', 3: 'Miércoles', 4: 'Jueves', 5: 'Viernes', 6: 'Sábado', 7: 'Domingo'}
    for elem in leer_output(numero_sol=2):
        if elem[1] == nombre_profesor:
            clases_profesor.append(elem)

    for i in range(len(clases_profesor)):
        clases_profesor[i][3] = dict_dias[clases_profesor[i][3]]
    clases_profesor = sorted(clases_profesor, key= lambda x: x[4])
    clases_profesor = sorted(clases_profesor, key= lambda x: x[3])
    for i in range(len(clases_profesor)):
        clases_profesor[i][3] = dict_dias_inverso[clases_profesor[i][3]]


    lista_auxiliar = []
    for i in dias:
        lista_dia = [i]
        for j in modulos:
            if clases_profesor == []:
                lista_dia.append(' ')
            elif i == clases_profesor[0][3] and j == int(clases_profesor[0][4]):
                lista_dia.append(f'{clases_profesor[0][0]} \n {clases_profesor[0][2]}')
                clases_profesor.pop(0)
            else:
                lista_dia.append(' ')
        lista_auxiliar.append(lista_dia)

    # print(dias)
    horario = PrettyTable([])
    horario.add_column('', modulos)
    # print(modulos)
    # print(horario)
    for i in range(len(dias)):
        # print(lista_auxiliar)
        horario.add_column(lista_auxiliar[i][0], lista_auxiliar[i][1:], align='c')
    horario.format = True   
    print(f'Horario del profesor {nombre_profesor}: ')   
    return horario

generar_horario_profesor(nombre_profesor='Nncolmnk')

Horario del profesor Nncolmnk: 


,Lunes,Martes,Miércoles,Jueves,Viernes
1,,,,,5A Matemáticas
2,4B Historia,,5A Matemáticas,,6C Artes_Visuales
3,,,7C Religión,,6C Artes_Visuales
4,,IVA G2,,IVA G2,
5,,IVA G2,,IVA G2,
6,7C Religión,4B Historia,,,
7,5A Matemáticas,,,5A Matemáticas,
8,,5A Matemáticas,4B Historia,,
9,,5A Matemáticas,,,
10,,,,,


### A continuación se presenta una idea donde se podría seleccionar ciertas componentes del horario y fijarlas o bloquearlas para una siguiente iteración

##### Una idea sería que, si se selecciona un bloque a la vez, uno puede fijarlo en esa posición o bloquearlo de esa posición

In [ ]:
asignaciones_vetadas = []
# ! Esta sería información que se obtiene clickeando el bloque
def vetar_asignacion(curso, profesor, clase, dia, modulo, asignaciones_vetadas):
    asignaciones_vetadas.append((curso, profesor, clase, dia, modulo))

def fijar_asignacion(curso, profesor, clase, dia, modulo, asignaciones_fijadas):
    asignaciones_fijadas.append((curso, profesor, clase, dia, modulo))

curso_vetado = '1A'
profesor_vetado = 'Pérez'
clase_vetada = 'Matemáticas'
dia_vetado = 'Lunes'
modulo_vetado = 2
asignaciones_vetadas.append((curso_vetado, profesor_vetado, clase_vetada, dia_vetado, modulo_vetado))
# * y después esto se agrega al modelo
model.addConstrs((x[c,p,k,d,j] == 0 for c,p,k,d,j in asignaciones_vetadas),name="RExtra1");

asignaciones_fijadas = []
curso_fijado = '1A'
profesor_fijado = 'González'
clase_fijada = 'Educación_Física'
dia_fijado = 'Martes'
modulo_fijado = 5
model.addConstrs((x[c,p,k,d,j] == 1 for c,p,k,d,j in asignaciones_fijadas),name="RExtra2");

##### También se puede querer mantener/vetar un comportamiento de dos partes del horario simultáneamente.

In [ ]:
# * Si se tienen dos cursos seleccionados se puede querer que en un siguiente horario estén juntos sí o sí. Esto obviamente no se aplicaría para el caso que sean clases que
# * necesariamente son consecutivas

combinaciones_vetadas = []

curso_a_tratar = '1A'
profesor1 = 'Rodríguez'
profesor2 = 'Pérez'
clase1 = 'Lenguaje'
clase2 = 'Matemáticas'
# ? Interesa realmente módulo y día

combinaciones_vetadas.append((curso_a_tratar, profesor1, profesor2, clase1, clase2))

# ! Para vetarlos
model.addConstrs((x[c,p1,k1,d,j] + x[c,p2,k2,d,z] <= 1 for c,p1,p2,k1,k2 in combinaciones_vetadas for d in dias for j,z in modulos_consecutivos),name="RExtra3");

def vetar_combinaciones(curso, profesor1, profesor2, clase1, clase2):
    combinaciones_vetadas.append((curso, profesor1, profesor2, clase1, clase2))

combinaciones_deseadas = []

curso_a_tratar = '1A'
profesor1 = 'González'
profesor2 = 'Pérez'
clase1 = 'Música'
clase2 = 'Matemáticas'
dia = 'Miércoles'
modulo1 = 1
modulo2 = 2

combinaciones_deseadas.append((curso_a_tratar,profesor1,profesor2,clase1,clase2,dia,modulo1,modulo2))

# * Para fijarlos
model.addConstrs((x[c,p1,k1,d,j1] + x[c,p2,k2,d,j2] == 2 for c,p1,p2,k1,k2,d,j1,j2 in combinaciones_deseadas),name="RExtra5");

In [ ]:
def actualizar_restricciones(modelo: gp.Model, asignaciones_fijadas: list, asignaciones_vetadas: list, combinaciones_vetadas: list):
    modelo.addConstrs((x[c,p,k,d,j] == 1 for c,p,k,d,j in asignaciones_fijadas), name="Fijadas")
    modelo.addConstrs((x[c,p,k,d,j] == 0 for c,p,k,d,j in asignaciones_vetadas), name="Vetadas")
    modelo.addConstrs((x[c,p1,k1,d,j] + x[c,p2,k2,d,z] <= 1 for c,p1,p2,k1,k2 in combinaciones_vetadas for d in dias for j,z in modulos_consecutivos),name="CombVetadas")
    return modelo

En el caso que no se tenga un profesor fijado y se quiera que el programa elija se tiene que tener una lista de profesores que puedan dictar esa clase (para ese curso)

In [ ]:
t = model.addVars(c,p,k)
# Esta variable t ayuda a que si no hay un profesor asignado se asigne uno dependiendo qué conviene más (en base a disponibilidad y otras restricciones)

In [ ]:
model.addConstrs((sum(x[c,p,k,d,j] for k in profesores_por_curso[c]['0'] for p in profesores_por_curso[c]['0'][k]) == 1 \
    for c in horarios for d in horarios[c] for j in horarios[c][d]),name="RNoProf1");

In [ ]:
model.addConstrs((M * t[c,p,k] >= sum(x[c,p,k,d,j] for d in horarios[c] for j in horarios[c][d]) \
    for c in horarios for k in profesores_por_curso[c]['0'] for p in profesores_por_curso[c]['0'][k]),name="RNoProf2")

In [ ]:
model.addConstrs((sum(t[c,p,k] for p in profesores_por_curso[c]['0'][k]) == 1 \
    for c in horarios for k in profesores_por_curso[c]['0'] ),name="RNoProf3");